In [1]:
# activate the project environment
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."));

  Activating project at `~/projects/julia-projects/dbscan`


In [2]:
using NearestNeighbors
using dbscan
using StaticArrays
using Dates
using CairoMakie
using LoggingExtras

In [3]:
fmt_logger = FormatLogger(stdout) do io, args
    # Write the module, level and message only
    println(io, args._module, " | ", "[", args.level, "] ", args.message)
end
debuglogger = MinLevelLogger(fmt_logger, Logging.Debug)

MinLevelLogger{FormatLogger, LogLevel}(FormatLogger(var"#11#12"(), VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(16) open, 0 bytes waiting)), VSCodeServer.io_send_callback), true), Debug)

In [4]:
with_logger(debuglogger) do
    @debug "test"
end

Main | [Debug] test


In [5]:
# points = [SVector{3}(rand(3)) for i in 1:1_000_000]
# points = map(i -> rand(3), 1:1_000_000)
# points = [rand(3) for _ in 1:1_000_000]
points = rand(SVector{2, Float64}, 1_000_000)

1000000-element Vector{SVector{2, Float64}}:
 [0.9909686338666918, 0.02325971722100384]
 [0.211705321380955, 0.23665454809990272]
 [0.6056921398961004, 0.6088515621171336]
 [0.6416826694639222, 0.055006212075434435]
 [0.8291302568448486, 0.12654606200191776]
 [0.32521167444729226, 0.9616001540279605]
 [0.3721133809813014, 0.199936149162262]
 [0.8510038039603874, 0.2811343008432735]
 [0.288593203916786, 0.6571844854388201]
 [0.4294179626427631, 0.5793165316821285]
 ⋮
 [0.46392048284460097, 0.03849489433794884]
 [0.9757557998006551, 0.08193662562396331]
 [0.3275099215894113, 0.6940169087789338]
 [0.6097248456348641, 0.029120983233418607]
 [0.29233828605306067, 0.8615416734505078]
 [0.6311523849938895, 0.5188879510813214]
 [0.6703455726202402, 0.4991423785769611]
 [0.10081940283925561, 0.1285668828979355]
 [0.6766865896553885, 0.27723439297098995]

In [6]:
cluster_radius = 0.0005
min_pts = 3

3

In [7]:
with_logger(debuglogger) do 
    clusters = dbscan.DBSCAN(points, cluster_radius, min_pts; n_threads = 1)
end

dbscan | [Debug] 390 milliseconds to build kdtree
dbscan | [Debug] performing range searches
dbscan | [Debug] starting thread 1
dbscan | [Debug] 735 milliseconds to process thread 1
dbscan | [Debug] empty period to merge clusters 1
dbscan | [Debug] updating labels to cluster roots
dbscan | [Debug] 30 milliseconds to update labels
dbscan | [Debug] DBSCAN completed


ValueIterator for a Dict{Int64, Vector{Int64}} with 21687 entries. Values:
  [141832, 505576, 636263, 781693]
  [441319, 472238, 812776, 826794, 996189]
  [17062, 115317, 749359, 963767]
  [405927, 453581, 721069, 812871]
  [395033, 595372, 740357, 800752, 932499]
  [20596, 660051, 740223, 827303]
  [174505, 217243, 389149, 616146, 852173]
  [305292, 386601, 507544, 894880]
  [90614, 128978, 423264, 445008, 457552]
  [207472, 247993, 573410, 782978, 959174]
  [261577, 347930, 850779, 997124]
  [32163, 101339, 288568, 320769, 627813, 987141]
  [315022, 404551, 501068, 649492]
  [223367, 317434, 386745, 675592, 838286, 964182]
  [385819, 390745, 411449, 733257, 915819]
  [366276, 476787, 913777, 960399]
  [547147, 595188, 632150, 769383]
  [24026, 97095, 333536, 672173, 810056]
  [104239, 593144, 730072, 752179]
  ⋮

In [8]:
clusters = with_logger(debuglogger) do 
    dbscan.DBSCAN(points, cluster_radius, min_pts; n_threads = 4)
end

dbscan | [Debug] 365 milliseconds to build kdtree
dbscan | [Debug] performing range searches
dbscan | [Debug] starting thread 2
dbscan | [Debug] starting thread 1
dbscan | [Debug] starting thread 3
dbscan | [Debug] starting thread 4
dbscan | [Debug] 208 milliseconds to process thread 4
dbscan | [Debug] 218 milliseconds to process thread 2
dbscan | [Debug] 219 milliseconds to process thread 3
dbscan | [Debug] 219 milliseconds to process thread 1
dbscan | [Debug] 1 millisecond to merge clusters 1
dbscan | [Debug] 1 millisecond to merge clusters 2
dbscan | [Debug] 1 millisecond to merge clusters 3
dbscan | [Debug] 1 millisecond to merge clusters 4
dbscan | [Debug] updating labels to cluster roots
dbscan | [Debug] 3 milliseconds to update labels
dbscan | [Debug] DBSCAN completed


ValueIterator for a Dict{Int64, Vector{Int64}} with 21687 entries. Values:
  [141832, 505576, 636263, 781693]
  [441319, 472238, 812776, 826794, 996189]
  [17062, 115317, 749359, 963767]
  [405927, 453581, 721069, 812871]
  [395033, 595372, 740357, 800752, 932499]
  [20596, 660051, 740223, 827303]
  [174505, 217243, 389149, 616146, 852173]
  [305292, 386601, 507544, 894880]
  [90614, 128978, 423264, 445008, 457552]
  [207472, 247993, 573410, 782978, 959174]
  [261577, 347930, 850779, 997124]
  [32163, 101339, 288568, 320769, 627813, 987141]
  [315022, 404551, 501068, 649492]
  [223367, 317434, 386745, 675592, 838286, 964182]
  [385819, 390745, 411449, 733257, 915819]
  [366276, 476787, 913777, 960399]
  [547147, 595188, 632150, 769383]
  [24026, 97095, 333536, 672173, 810056]
  [104239, 593144, 730072, 752179]
  ⋮

In [9]:
# for i in eachindex(clusters)
#     if clusters[i] != 0 
#         println(i, ' ', clusters[i])
#         j = dbscan.find_root(i, clusters)
#         clusters[i] = j
#     end
# end

In [10]:
# for i in eachindex(clusters)
#     if clusters[i] != 0 
#         println(i, ' ', clusters[i])
#     end
# end

In [11]:
minimum(map(length, clusters))

4

In [12]:
tree = KDTree(points)

KDTree{SVector{2, Float64}, Euclidean, Float64, SVector{2, Float64}}
  Number of points: 1000000
  Dimensions: 2
  Metric: Euclidean(0.0)
  Reordered: true

In [13]:
# inrange(tree, points[263613], cluster_radius)

In [14]:
function draw_circle!(ax, x0, y0, r)
    t = 0:0.01:2pi
    x = x0 .+ r .* cos.(t)
    y = y0 .+ r .* sin.(t)
    lines!(ax, x, y; color = :black, alpha = 0.1)
end

draw_circle! (generic function with 1 method)

In [15]:
# f = Figure()
# ax = Axis(f[1, 1])

# scatter!(points; color = :gray, markersize = 2.0)
# for (id, cluster) in enumerate(clusters)
#     scatter!(points[cluster])
#     for (x0, y0) in points[cluster]
#         draw_circle!(ax, x0, y0, cluster_radius)
#     end
# end

# f

In [16]:
# wrap timings in a function to avoid global variables causing problems
function run_tests(N; n_threads = 1)
    # times = zeros(N)
    times = []
    for i in 1:N
        points = rand(SVector{3, Float64}, 1_000_000)
        t0 = now()
        labels = dbscan.DBSCAN(points, 0.0005, 3, n_threads = n_threads)
        tf = now()
        push!(times, canonicalize(tf - t0))
    end
    return times
end

run_tests (generic function with 1 method)

In [17]:
times_1 = run_tests(10; n_threads = 1)

10-element Vector{Any}:
 1 second, 739 milliseconds
 1 second, 523 milliseconds
 1 second, 371 milliseconds
 1 second, 333 milliseconds
 1 second, 548 milliseconds
 1 second, 498 milliseconds
 1 second, 277 milliseconds
 1 second, 297 milliseconds
 1 second, 445 milliseconds
 1 second, 320 milliseconds

In [18]:
times_2 = run_tests(10; n_threads = 2)

10-element Vector{Any}:
 1 second, 92 milliseconds
 972 milliseconds
 1 second, 119 milliseconds
 935 milliseconds
 972 milliseconds
 919 milliseconds
 1 second, 4 milliseconds
 959 milliseconds
 1 second, 87 milliseconds
 982 milliseconds

In [19]:
times_4 = run_tests(10; n_threads = 4)

10-element Vector{Any}:
 781 milliseconds
 854 milliseconds
 948 milliseconds
 800 milliseconds
 973 milliseconds
 832 milliseconds
 968 milliseconds
 802 milliseconds
 787 milliseconds
 791 milliseconds

In [20]:
# smaller dataset for plotting